In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image, ImageTk

In [27]:
model = load_model('mnist-4-7.h5')   #7 is the best till now

def predict_digit(img):
    #resize image to 75x75 pixels
    img = img.resize((100, 100))
    
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    
#     print('Image to be predicted...')
#     plt.imshow(img, cmap = 'gray')
#     plt.show()
    
    #reshaping to support our model input and normalizing
    img = img.reshape(1, 100, 100, 1).astype('float32')
    img = img/255.0
    

    #predicting the class
    res = model.predict([img])
    boxes = res.flatten().reshape((10, 2))

    return boxes

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0
        
        # Creating elements
        self.title('Handwritten Digit Recognition and Class Detection')
        self.canvas1 = tk.Canvas(self, width=500, height=500, bg = 'black', cursor="cross")
        self.classify_btn = tk.Button(self, text = "Recognise", command = self.classify_handwriting)   
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
       
    # Grid structure
        self.canvas1.grid(row=0, column=0, pady=0, sticky=W, rowspan = 2, columnspan = 2)
        self.classify_btn.grid(row=2, column=1, pady=2, padx = 2)
        self.button_clear.grid(row=2, column=0, pady=2)
        
        self.canvas1.bind("<B1-Motion>", self.draw_lines) # event handler for mouse events

    def clear_all(self):
        self.canvas1.delete("all")
        
    def classify_handwriting(self):
        # code to convert drawing on canvas to an image
        HWND = self.canvas1.winfo_id()        # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        self.im = ImageGrab.grab(rect)
    
        # predict what the image is...
        boxes_10 = predict_digit(self.im)
        boxes_list = []
        labels = []
        div = 100
        for n, each_box in enumerate(boxes_10):
            if (each_box[0] > 0.20 and each_box[1] >= 0.0):
                    labels.append(n)
                    box = [each_box[1], (each_box[0]*100 - 28)/div, (each_box[1]*100 + 28)/div, each_box[0]]
                    boxes_list.append(box)

        w, h = self.im.size
        
        for each_box in boxes_list:
            each_box[0] = each_box[0] * h
            each_box[1] = each_box[1] * w
            each_box[2] = each_box[2] * h
            each_box[3] = each_box[3] * w

        self.img = ImageTk.PhotoImage(self.im)
        self.canvas1.create_image((w/2, h/2), image=self.img, anchor = "center")
        
        for label, each_box in zip(labels, boxes_list):
            self.canvas1.create_rectangle(each_box[1], each_box[0], each_box[3], each_box[2], 
                                          outline = 'red')
            if each_box[0]<=20:
                self.canvas1.create_text(each_box[1]+5, each_box[2] +15, fill = 'red', text = label, 
                                     font="Times 20 italic bold")
            elif each_box[1]<=20:
                self.canvas1.create_text(each_box[3]-15, each_box[0]-15, fill = 'red', text = label, 
                                     font="Times 20 italic bold")
            else:    
                self.canvas1.create_text(each_box[1] +5, each_box[0] -15, fill = 'red', text = label, 
                                     font="Times 20 italic bold") 


    def draw_lines(self, event):
        # draw on the canvas
        self.x = event.x
        self.y = event.y
        r= 6                              # control the width of strokes
        self.canvas1.create_oval(self.x+r, self.y+r, self.x - r, self.y - r, fill='white',outline = 'white')
       
app = App()
mainloop()
